# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# 作業1
* 觀察範例，在房價預測中調整標籤編碼(Label Encoder) / 獨熱編碼 (One Hot Encoder) 方式，  
對於線性迴歸以及梯度提升樹兩種模型，何者影響比較大?

> 線性迴歸對Label Encoder與One Hot Encoder結果差距非常大。可能原因是在做One Hot Encoder時並沒有剔除Dummy Variable多餘的欄位，會造成嚴重的VIF問題。因而導致準確度下降。

In [12]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

data_path = 'data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [13]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} Numeric Features : {object_features}\n')

# 只留類別型欄位
df = df[object_features]
df = df.fillna('None')
train_num = train_Y.shape[0]
df.head()

5 Numeric Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


# 作業2
* 鐵達尼號例題中，標籤編碼 / 獨熱編碼又分別對預測結果有何影響? (Hint : 參考今日範例)

In [14]:
# 標籤編碼 + 羅吉斯迴歸
df_label = pd.DataFrame()
for c in df.columns:
    df_label[c] = LabelEncoder().fit_transform(df[c])
df_label.head()

,Name,Sex,Ticket,Cabin,Embarked
0,155,1,720,185,3
1,286,0,816,106,0
2,523,0,914,185,3
3,422,0,65,70,3
4,22,1,649,185,3


In [31]:
estimator = LogisticRegression()
train_X = df_label[:train_num]
start = time.time()
print(f'shape: {train_X.shape}')
print(f'score: {cross_val_score(estimator, train_X, train_Y, cv = 5).mean()}')
print(f'time: {time.time()-start}')

shape: (891, 5)
score: 0.780004837244799
time: 0.02094554901123047


In [36]:
# 獨熱編碼 + 羅吉斯迴歸
from sklearn.preprocessing import OneHotEncoder
df_onehot = df_label.values
df_onehot = OneHotEncoder().fit_transform(df_onehot).toarray()

In [38]:
df_onehot = pd.DataFrame(df_onehot)

In [39]:
df_onehot.head()

,0,1,2,3,4,5,6,7,8,9,...,2419,2420,2421,2422,2423,2424,2425,2426,2427,2428
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [40]:
estimator = LogisticRegression()
train_X = df_onehot[:train_num]
start = time.time()
print(f'shape: {train_X.shape}')
print(f'score: {cross_val_score(estimator, train_X, train_Y, cv = 5).mean()}')
print(f'time: {time.time()-start}')

shape: (891, 2429)
score: 0.8013346043513216
time: 0.10369467735290527


In [44]:
# 用get_dummies whithout dropfirst
df_onehot_get_dummies = pd.get_dummies(df)
estimator = LogisticRegression()
train_X = df_onehot_get_dummies[:train_num]
start = time.time()
print(f'shape: {train_X.shape}')
print(f'score: {cross_val_score(estimator, train_X, train_Y, cv = 5).mean()}')
print(f'time: {time.time()-start}')

shape: (891, 2429)
score: 0.8013346043513216
time: 0.13062286376953125


In [45]:
# 用get_dummies whith dropfirst
df_onehot_get_dummies = pd.get_dummies(df, drop_first=True)
estimator = LogisticRegression()
train_X = df_onehot_get_dummies[:train_num]
start = time.time()
print(f'shape: {train_X.shape}')
print(f'score: {cross_val_score(estimator, train_X, train_Y, cv = 5).mean()}')
print(f'time: {time.time()-start}')

shape: (891, 2424)
score: 0.8046928367284479
time: 0.13065075874328613
